## Dietary Food Recomender System

In [39]:
# Import Library
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
# Load Dataset
df = pd.read_csv('after_encoding.csv')
df.head()

C:\Users\jason\AppData\Local\Temp\ipykernel_17696\156284050.py:2: DtypeWarning: Columns (7,117,1309,1340,1494,1964,2306,2330,2628,2799,2800,2816,3330,3605,3869,4069,4071,4073,4214,4233,4238,4279,4501,4724,4727,4797,4806,4816,4875,5441,5477,5483,5566,6024,6077,6079,6091,6104,6125,6177,6278,6632,6640,6950,6953,6966,6969,6991,6996,7223,7243,7304,7333,7389,8018,8106,9213) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('after_encoding.csv')


,name,ingredients,steps,calories,carbs,protein,fat,servings,image_url,link,...,"zucchini, cut into small chunks","zucchini, cut into spirals using a spiral slicer","zucchini, diced","zucchini, grated and squeezed dry","zucchini, peeled and finely chopped","zucchini, shredded","zucchini, sliced in half vertically, then sliced crosswise","zucchini, thinly sliced",zucchinis,"zucchinis, thinly sliced lengthwise"
0,air fryer lemon garlic parmesan chicken,"[{'quantity': '1 1/2', 'unit': 'pounds', 'name...",Gather all ingredients.. Preheat an air fryer ...,365,8g,0,17g,4,https://www.allrecipes.com/thmb/5nJvgXENSeFx82...,https://www.allrecipes.com/air-fryer-lemon-gar...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0
1,air fryer s’mores,"[{'quantity': '1', 'unit': 'sleeve', 'name': '...",Preheat an air fryer to 380 degrees F (193 deg...,143,20g,0,6g,10,https://www.allrecipes.com/thmb/_EDaiFt0gIGQOL...,https://www.allrecipes.com/air-fryer-s-mores-r...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0
2,air fryer baked yams,"[{'quantity': '1', 'unit': 'pieces', 'name': '...",Preheat an air fryer to 400 degrees F (200 deg...,283,62g,0,3g,1,https://www.allrecipes.com/thmb/156WNgRfzvGn-s...,https://www.allrecipes.com/air-fryer-baked-yam...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0
3,lemon garlic butter chicken spiedini,"[{'quantity': '1/2', 'unit': 'cup', 'name': 'e...","Whisk together olive oil, wine, 2 tablespoons ...",636,21g,0,41g,6,https://cdn.jwplayer.com/v2/media/ahbYMLcr/thu...,https://www.allrecipes.com/lemon-garlic-butter...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0
4,air fryer grilled pimento cheese,"[{'quantity': '4', 'unit': 'slices', 'name': '...",Preheat the air fryer to 370 degrees F (188 de...,902,108g,0,40g,2,https://www.allrecipes.com/thmb/cdL3DKZH3beUk5...,https://www.allrecipes.com/air-fryer-grilled-p...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0


In [41]:
# Select columns with float64 and int64 types
numeric_df = df.select_dtypes(include=['float64', 'int64'])
numeric_df = numeric_df.drop(columns=['calories', 'protein'])
# Display the result
print(numeric_df)

      (1 1/4-inch-thick) tenderloin steaks  \
0                                        0   
1                                        0   
2                                        0   
3                                        0   
4                                        0   
...                                    ...   
2905                                     0   
2906                                     0   
2907                                     0   
2908                                     0   
2909                                     0   

      (1 inch thick) filet mignon steaks  \
0                                      0   
1                                      0   
2                                      0   
3                                      0   
4                                      0   
...                                  ...   
2905                                   0   
2906                                   0   
2907                                   0   
2908   

In [42]:
# Select All the Ingredients Columns
ingredients = numeric_df

# Compute Cosine Similarity Matrix
similarity = cosine_similarity(ingredients)

In [43]:
# Base Recommendation Function
def recommend_similar_recipes(name, filter_func=None, top_n=10):
    # Check if the Recipe Exists
    if name not in df['name'].values:
        return f"Recipe '{name}' not found."
    
    # Get the Index of the Recipe
    recipe_id = df[df['name'] == name].index[0]
    
    # Get Similarity Score for the Given Recipe
    sim_scores = list(enumerate(similarity[recipe_id]))
    
    # Sort Scores in Descending Order of Similarity
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Exclude the recipe itself
    sim_scores = sim_scores[1:]
    
    # Apply Calorie Filter if Provided
    if filter_func:
        sim_scores = [score for score in sim_scores if filter_func(score[0])]
        
    # Get the top_n Indices and Scores after Filtering
    sim_scores = sim_scores[:top_n]
    recipe_indices = [i[0] for i in sim_scores]
    recipe_similarities = [i[1] for i in sim_scores]
    
    # Prepare the result as a list of tuples: (recipe_name, similarity_score)
    recommended_recipes = [
        (df['name'].iloc[idx], similarity) 
        for idx, similarity in zip(recipe_indices, recipe_similarities)
    ]
    
    return recommended_recipes

In [44]:
# Recommendation Function for Higher Recommended Recipes
def recommend_higher_calorie_recipe(name, top_n=10):
    target_calorie = df.loc[df['name'] == name, 'calories'].values[0]
    return recommend_similar_recipes(
        name,
        filter_func=lambda idx: df.iloc[idx, df.columns.get_loc('calories')] > target_calorie,
        top_n=top_n
    )

# Recommendation Function for Lower Recommended Recipes
def recommend_lower_calorie_recipe(name, top_n=10):
    target_calorie = df.loc[df['name'] == name, 'calories'].values[0]
    return recommend_similar_recipes(
        name,
        filter_func=lambda idx: df.iloc[idx, df.columns.get_loc('calories')] < target_calorie,
        top_n=top_n
    )

In [45]:
# Test the recommender systems
print("Similar recipes with higher calories:")
print(recommend_higher_calorie_recipe('air fryer lemon garlic parmesan chicken', top_n=5))

print("\nSimilar recipes with lower calories:")
print(recommend_lower_calorie_recipe('air fryer lemon garlic parmesan chicken', top_n=5))

Similar recipes with higher calories:
[('lemon garlic butter chicken spiedini', 0.9312995286526017), ('air fryer chicken tenders', 0.07435054969371101), ('air fryer cheese curds', 0.05979712872469897), ('teriyaki chicken cordon bleu', 0.05078396591492308), ('baked buffalo chicken mac and cheese', 0.038443276362861926)]

Similar recipes with lower calories:
[('air fryer parmesan chicken bites', 0.07445849689419802), ('air fryer crispy pickle chips with creole dipping sauce', 0.0696862588505459), ('quick crispy parmesan chicken breasts', 0.06853700004240314), ('air fryer vegan buffalo tofu bites', 0.06098762556493499), ('fresh salmon burgers in the air fryer', 0.03175273028919799)]


In [46]:
# Locate the rows of the specific recipes by their names
recipes_to_check = ['air fryer lemon garlic parmesan chicken','lemon garlic butter chicken spiedini', 'air fryer chicken tenders', 'air fryer cheese curds', 'teriyaki chicken cordon bleu', 'baked buffalo chicken mac and cheese']
selected_recipes = df.loc[df['name'].isin(recipes_to_check)]

In [47]:
# Display the selected rows
print("\nSelected high calories:")
df_selected_recipes = pd.DataFrame(selected_recipes)
df_selected_recipes


Selected high calories:


,name,ingredients,steps,calories,carbs,protein,fat,servings,image_url,link,...,"zucchini, cut into small chunks","zucchini, cut into spirals using a spiral slicer","zucchini, diced","zucchini, grated and squeezed dry","zucchini, peeled and finely chopped","zucchini, shredded","zucchini, sliced in half vertically, then sliced crosswise","zucchini, thinly sliced",zucchinis,"zucchinis, thinly sliced lengthwise"
0,air fryer lemon garlic parmesan chicken,"[{'quantity': '1 1/2', 'unit': 'pounds', 'name...",Gather all ingredients.. Preheat an air fryer ...,365,8g,0,17g,4,https://www.allrecipes.com/thmb/5nJvgXENSeFx82...,https://www.allrecipes.com/air-fryer-lemon-gar...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0
3,lemon garlic butter chicken spiedini,"[{'quantity': '1/2', 'unit': 'cup', 'name': 'e...","Whisk together olive oil, wine, 2 tablespoons ...",636,21g,0,41g,6,https://cdn.jwplayer.com/v2/media/ahbYMLcr/thu...,https://www.allrecipes.com/lemon-garlic-butter...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0
40,air fryer chicken tenders,"[{'quantity': None, 'unit': None, 'name': 'coo...",Preheat air fryer to 350 degrees F (175 degree...,509,33g,0,31g,4,https://www.allrecipes.com/thmb/gVasI5bp_6qAmC...,https://www.allrecipes.com/air-fryer-chicken-t...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0
46,air fryer cheese curds,"[{'quantity': '3', 'unit': 'cloves', 'name': '...","For Aioli: Combine garlic cloves, olive oil, a...",944,81g,0,53g,4,https://www.allrecipes.com/thmb/bpDQGQ5MqM-olL...,https://www.allrecipes.com/air-fryer-cheese-cu...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0
1269,baked buffalo chicken mac and cheese,"[{'quantity': '', 'unit': '', 'name': 'cooking...",Preheat oven to 350 degrees F (175 degrees C)....,917,67g,0,53g,8,https://www.allrecipes.com/thmb/V8NshK1PrRxUE0...,https://www.allrecipes.com/recipe/236942/baked...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0
2165,teriyaki chicken cordon bleu,"[{'quantity': '4', 'unit': 'large', 'name': 's...",Place the chicken thighs between two sheets of...,501,32g,0,25g,4,https://www.allrecipes.com/thmb/XHqfRQe_Ad2JN2...,https://www.allrecipes.com/recipe/215008/teriy...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0


In [48]:
print("\nSelected calories:")
print(selected_calories)


Selected calories:
                                         name  \
0     air fryer lemon garlic parmesan chicken   
3        lemon garlic butter chicken spiedini   
40                  air fryer chicken tenders   
46                     air fryer cheese curds   
1269     baked buffalo chicken mac and cheese   
2165             teriyaki chicken cordon bleu   

                                            ingredients  \
0     [{'quantity': '1 1/2', 'unit': 'pounds', 'name...   
3     [{'quantity': '1/2', 'unit': 'cup', 'name': 'e...   
40    [{'quantity': None, 'unit': None, 'name': 'coo...   
46    [{'quantity': '3', 'unit': 'cloves', 'name': '...   
1269  [{'quantity': '', 'unit': '', 'name': 'cooking...   
2165  [{'quantity': '4', 'unit': 'large', 'name': 's...   

                                                  steps  calories carbs  \
0     Gather all ingredients.. Preheat an air fryer ...       365    8g   
3     Whisk together olive oil, wine, 2 tablespoons ...       636   21g 